In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torchvision


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r '/content/drive/MyDrive/rotation_classification' '/content'

In [4]:
import os
try:
  os.mkdir('/content/train')
  os.mkdir('/content/val')
  os.mkdir('/content/test')
except OSError as error:
  print(error)

In [5]:
!cp '/content/rotation_classification/train (2).zip' '/content/train/train.zip'
!unzip '/content/train/train.zip' -d '/content/train'
!rm -f '/content/train/train.zip'

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/5499.jpg  
  inflating: /content/train/55.jpg   
  inflating: /content/train/550.jpg  
  inflating: /content/train/5500.jpg  
  inflating: /content/train/5501.jpg  
  inflating: /content/train/5502.jpg  
  inflating: /content/train/5503.jpg  
  inflating: /content/train/5504.jpg  
  inflating: /content/train/5505.jpg  
  inflating: /content/train/5506.jpg  
  inflating: /content/train/5507.jpg  
  inflating: /content/train/5508.jpg  
  inflating: /content/train/5509.jpg  
  inflating: /content/train/551.jpg  
  inflating: /content/train/5510.jpg  
  inflating: /content/train/5511.jpg  
  inflating: /content/train/5512.jpg  
  inflating: /content/train/5513.jpg  
  inflating: /content/train/5514.jpg  
  inflating: /content/train/5515.jpg  
  inflating: /content/train/5516.jpg  
  inflating: /content/train/5517.jpg  
  inflating: /content/train/5518.jpg  
  inflating: /content/train/5519.jpg  
  inflating: /co

In [6]:
!cp '/content/rotation_classification/val (2).zip' '/content/val/val.zip'
!unzip '/content/val/val.zip' -d '/content/val'
!rm -f '/content/val/val.zip'

Archive:  /content/val/val.zip
  inflating: /content/val/0.jpg      
  inflating: /content/val/1.jpg      
  inflating: /content/val/10.jpg     
  inflating: /content/val/100.jpg    
  inflating: /content/val/1000.jpg   
  inflating: /content/val/1001.jpg   
  inflating: /content/val/1002.jpg   
  inflating: /content/val/1003.jpg   
  inflating: /content/val/1004.jpg   
  inflating: /content/val/1005.jpg   
  inflating: /content/val/1006.jpg   
  inflating: /content/val/1007.jpg   
  inflating: /content/val/1008.jpg   
  inflating: /content/val/1009.jpg   
  inflating: /content/val/101.jpg    
  inflating: /content/val/1010.jpg   
  inflating: /content/val/1011.jpg   
  inflating: /content/val/1012.jpg   
  inflating: /content/val/1013.jpg   
  inflating: /content/val/1014.jpg   
  inflating: /content/val/1015.jpg   
  inflating: /content/val/1016.jpg   
  inflating: /content/val/1017.jpg   
  inflating: /content/val/1018.jpg   
  inflating: /content/val/1019.jpg   
  inflating: /conte

In [7]:
!cp '/content/rotation_classification/val.csv' '/content/val.csv'
!cp '/content/rotation_classification/train (1).csv' '/content/train.csv'

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# classes = np.asarray([i for i in range(1,361)])
# enc = OneHotEncoder(sparse=False)
# enc.fit(classes.reshape(len(classes),1))

In [ ]:
# enc.categories_[0:5]

In [ ]:
# temp_dat = enc.transform([[360]])
# print(temp_dat)
# print(np.argmax(temp_dat)+1)

In [ ]:
# temp_dat = enc.transform([[1]])
# print(temp_dat)
# print(np.argmax(temp_dat)+1)

In [8]:
train_csv = pd.read_csv('/content/train.csv')
dummy = train_csv.values
print(dummy[0:5])
dummy1 = dummy[:,1]
train_data = dummy1
print(train_data[0:5])

[[  0 141]
 [  1  53]
 [  2  45]
 [  3  48]
 [  4  14]]
[141  53  45  48  14]


In [ ]:
# print(train_data[0])
# print(np.argmax(train_data[0])+1)

In [9]:
val_csv = pd.read_csv('/content/val.csv')
vals = val_csv.values
vals1 = vals[:,1]
val_data = vals1
print(val_data[0:5])

[141  53  45  48  14]


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
from PIL import Image 


class rotationclass(Dataset):

  def __init__(self,csv_data,root_dir,transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.data = csv_data

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    img_path = self.root_dir +'/'+ str(idx) + '.jpg'
    image = Image.open(img_path)
    label = int(self.data[idx])
    
    if self.transform:
      tensor_image = self.transform(image)

    return tensor_image,label



In [12]:
import torchvision.models as models
resnet18 = models.resnet18()
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
resnet18.fc = torch.nn.Linear(512,360)
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
resnet18 = resnet18.to(device)

batch_size = 32
train_set = rotationclass(train_data,'/content/train',transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                                              ]))
train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)

val_set = rotationclass(val_data,'/content/val',transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                                                 ]))
val_loader = torch.utils.data.DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [15]:
def evaluation(dataloader):
  total,correct = 0,0
  for data in dataloader:
    inputs,labels = data
    inputs,labels = inputs.to(device),labels.to(device)
    outputs = resnet18(inputs)
    _,pred = torch.max(outputs.data,1) 
    pred = pred + 1
    total += labels.size(0)
    correct += (pred==labels).sum().item()
  
  return 100*correct/total

In [16]:
import torch.optim as optim
import torch.nn as nn
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet18.parameters())

In [19]:
from tqdm import tqdm
loss_arr = []
loss_epoch_arr = []
max_epochs = 15

for epoch in tqdm(range(max_epochs)):
  for i, data in enumerate(train_loader,0):
    inputs, labels = data
    inputs,labels = inputs.to(device),labels.to(device)
    opt.zero_grad()

    outputs = resnet18(inputs)
    #print(outputs.shape)
    #_,pred_labels = torch.max(outputs.data,1)
    #pred_labels = pred_labels + 1
    #print(pred_labels)
    #print(labels)
    loss = loss_fn(outputs,labels-1)
    loss.backward()
    opt.step()

    loss_arr.append(loss.item())

  PATH = '/content/' + 'model_' + str(epoch) + '.pt'
  
  
  loss_epoch_arr.append(loss.item())

  print('Epoch = %d/%d, Val Acc: %0.2f, Train Acc: %0.2f '%(epoch,max_epochs,
        evaluation(val_loader),evaluation(train_loader)))
  torch.save({
            'epoch': epoch,
            'model_state_dict': resnet18.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss.item(),
            }, PATH)

plt.plot(loss_epoch_arr)
plt.show()

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch = 0/15, Val Acc: 0.42, Train Acc: 0.36 


  7%|▋         | 1/15 [18:36<4:20:29, 1116.38s/it]

Epoch = 1/15, Val Acc: 0.42, Train Acc: 0.36 


 13%|█▎        | 2/15 [37:24<4:02:40, 1120.03s/it]

Epoch = 2/15, Val Acc: 0.42, Train Acc: 0.36 


 20%|██        | 3/15 [56:09<3:44:15, 1121.26s/it]

KeyboardInterrupt: ignored

In [17]:
new_model = models.resnet18()
new_model.fc = torch.nn.Linear(512,360)
checkpoint = torch.load('/content/model_6.pt')
new_model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
new_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [47]:
!cp '/content/drive/MyDrive/rotation_classification/test (2).zip' '/content/rotation_classification/test (2).zip'

In [48]:
!cp '/content/rotation_classification/test (2).zip' '/content/test/test.zip'
!unzip '/content/test/test.zip' -d '/content/test'
!rm -f '/content/test/test.zip'

Streaming output truncated to the last 5000 lines.
  inflating: /content/test/5499.jpg  
  inflating: /content/test/55.jpg    
  inflating: /content/test/550.jpg   
  inflating: /content/test/5500.jpg  
  inflating: /content/test/5501.jpg  
  inflating: /content/test/5502.jpg  
  inflating: /content/test/5503.jpg  
  inflating: /content/test/5504.jpg  
  inflating: /content/test/5505.jpg  
  inflating: /content/test/5506.jpg  
  inflating: /content/test/5507.jpg  
  inflating: /content/test/5508.jpg  
  inflating: /content/test/5509.jpg  
  inflating: /content/test/551.jpg   
  inflating: /content/test/5510.jpg  
  inflating: /content/test/5511.jpg  
  inflating: /content/test/5512.jpg  
  inflating: /content/test/5513.jpg  
  inflating: /content/test/5514.jpg  
  inflating: /content/test/5515.jpg  
  inflating: /content/test/5516.jpg  
  inflating: /content/test/5517.jpg  
  inflating: /content/test/5518.jpg  
  inflating: /content/test/5519.jpg  
  inflating: /content/test/552.jpg   

In [ ]:
# import os
# test_images = os.listdir('/content/test')
# print(test_images[0:5])

['5266.jpg', '2729.jpg', '7586.jpg', '4675.jpg', '9178.jpg']


In [ ]:
# transformation = transforms.Compose([
#           transforms.ToTensor(),
#           transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
#                                                                  ])

In [ ]:
# out_final = []
# new_model = new_model.to(device)
# # curiosity = 0
# # perseverance = 1
# for images in tqdm(test_images):
#   index = images.split('.')[0]
#   img_path = '/content/test/'+ images
#   image = Image.open(img_path)
#   #image = np.expand_dims(image,axis=0)
#   image = transformation(image)
#   #image = transforms.Normalize(image,mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#   image = image.unsqueeze(0)
#   image = image.to(device)
#   out = new_model(image)
#   _,pred = torch.max(out.data,1)
#   angle = pred.item()
#   angle = angle + 1
#   arr = [int(index),angle]
#   out_final.append(arr)

  


In [ ]:
# print(out_final[0:5])

In [ ]:
# dataset = pd.DataFrame(out_final,columns=['ImageID','label'])
# dataset.head()

In [ ]:
# dataset.to_csv('output_mars_angle_classification.csv',index=False) 

In [49]:
#Batch normalization problem

class rotationclass_test(Dataset):

  def __init__(self,root_dir,transform=None):
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return 10000

  def __getitem__(self,idx):
    img_path = self.root_dir +'/'+ str(idx) + '.jpg'
    image = Image.open(img_path)
    image_id = idx
    
    if self.transform:
      tensor_image = self.transform(image)

    return tensor_image,image_id

In [50]:
test_set = rotationclass_test('/content/test/',transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                                              ]))

test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False)

In [51]:
from tqdm import tqdm
out_final = []
out_index = []
new_model = new_model.to(device)
for i, data in tqdm(enumerate(test_loader,0)):
  image,index = data
  image = image.to(device)
  with torch.no_grad():
    out = new_model(image)
    _,pred = torch.max(out.data,1)
    pred = pred + 1
    out_final.append(pred.cpu().numpy())
    out_index.append(index.cpu().numpy())

313it [00:39,  7.85it/s]


In [52]:
print(out_final[0:5])

[array([195, 272, 324, 231, 298,  56, 101,  15, 308,  62, 201, 111, 122,
       317, 200,  60,  70,  21,   5, 158,  38, 100, 218, 272,   3,  27,
        34,  69, 135, 103, 339, 335]), array([233,  61, 171, 318,  43, 204, 340, 108, 181, 319,  91, 166, 194,
        41, 168,  19, 289, 206, 329, 102, 113, 233, 255, 213, 175,  64,
       205, 315, 218,   6, 274, 153]), array([217, 243, 220, 262,  18, 241, 115,  22, 102,  80,   2, 155,  66,
       134, 291,  22, 111, 323,   4, 239, 337, 216, 203, 249, 163, 344,
       290, 108, 258, 139,  75, 157]), array([167, 173, 322, 186,  68, 128, 279, 143,  38, 292, 123, 314,  78,
       136, 125, 255,  84, 188,  51, 289, 102, 273, 262, 211, 125, 287,
       247, 299, 343,  29, 106,  29]), array([144,  38,  67, 192,  94,  16,  97, 166, 185, 315,  65, 111, 205,
        20, 160, 243, 313, 129, 323, 175,  19, 241,  59, 352, 274, 222,
        72, 192,  99, 183, 270, 273])]


In [53]:
print(out_index[0:5])

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), array([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]), array([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]), array([ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127]), array([128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159])]


In [54]:
out_result = []
for index,val in zip(out_index,out_final):
  for ind,v in zip(index,val):
    arr = [ind,v]
    out_result.append(arr)


In [55]:
print(out_result[0:5])

[[0, 195], [1, 272], [2, 324], [3, 231], [4, 298]]


In [56]:
dataset = pd.DataFrame(out_result,columns=['ImageID','label'])
dataset.head()

,ImageID,label
0,0,195
1,1,272
2,2,324
3,3,231
4,4,298


In [57]:
dataset.to_csv('output_mars_angle_classification.csv',index=False) 